In [ ]:
pip install duckdb pandas matplotlib seaborn statsmodels scipy --quiet

In [6]:
import duckdb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np

# Connect to the DuckDB database
con = duckdb.connect(database='gov-contracts.db', read_only=True)

In [ ]:
# Query to select relevant columns
query = '''
SELECT 
    department_ind_agency,
    cgac,
    sub_tier,
    fpds_code,
    office,
    aac_code,
    posteddate,
    type,
    basetype,
    popstreetaddress,
    popcity,
    popstate,
    popzip,
    popcountry,
    active,
    awardnumber,
    awarddate,
    award,
    awardee,
    state,
    city,
    zipcode,
    countrycode
FROM 
    ContractOpportunitiesFull
WHERE
    award IS NOT NULL
'''

# Execute the query and load data into a DataFrame
df = con.execute(query).fetchdf()
df

In [ ]:
# Load data# Check for missing values
missing_values = df.isnull().sum()
print(missing_values)

In [ ]:
# For simplicity, drop rows with missing values in key columns
df = df.dropna(subset=['award', 'cgac', 'awardee', 'state'])
df

In [ ]:
# Convert 'awarddate' and 'posteddate' to datetime
df['awarddate'] = pd.to_datetime(df['awarddate'], errors='coerce')
df['posteddate'] = pd.to_datetime(df['posteddate'], errors='coerce')

# Convert 'award' to numeric
df['award'] = pd.to_numeric(df['award'], errors='coerce')

# Drop rows with invalid dates or award amounts
df = df.dropna(subset=['awarddate', 'posteddate', 'award'])
df

In [ ]:
# List of categorical variables to encode
categorical_vars = ['department_ind_agency', 'sub_tier', 'type', 'basetype', 'popstate', 'popcountry', 'active', 'awardee', 'state', 'countrycode']

# Use one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_vars, drop_first=True)
df_encoded

In [ ]:
# Histogram of award amounts
plt.figure(figsize=(10, 6))
sns.histplot(df['award'], bins=50, kde=True)
plt.title('Distribution of Award Amounts')
plt.xlabel('Award Amount')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Top 10 states by total award amount
state_awards = df.groupby('state')['award'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 6))
state_awards.plot(kind='bar')
plt.title('Top 10 States by Total Award Amount')
plt.xlabel('State')
plt.ylabel('Total Award Amount')
plt.show()

In [ ]:
# Top 10 awardees by total award amount
awardee_awards = df.groupby('awardee')['award'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 6))
awardee_awards.plot(kind='bar')
plt.title('Top 10 Awardees by Total Award Amount')
plt.xlabel('Awardee')
plt.ylabel('Total Award Amount')
plt.show()

In [17]:
# Dependent variable
Y = df_encoded['award']

# Independent variables (selecting a subset for simplicity)
X = df_encoded[['cgac'] + [col for col in df_encoded.columns if 'state_' in col or 'type_' in col]]

# Add constant term
X = sm.add_constant(X)

In [ ]:
Xs

In [ ]:
Y

In [ ]:
# Example data
# Replace these with your actual data
Y = np.array([1, 2, 3, 4, 5])
X = np.array([1, 2, 3, 4, 5])

# Add a constant to the independent variable matrix
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(Y, X).fit()

# Summary of the model
print(model.summary())

In [ ]:
# Plot residuals
plt.figure(figsize=(10, 6))
sns.residplot(x=model.fittedvalues, y=model.resid, lowess=True)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()

In [ ]:
import scipy.stats as stats

# QQ plot
sm.qqplot(model.resid, line='s')
plt.title('QQ Plot of Residuals')
plt.show()